In [1]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import pandas as pd
import re
import numpy as np

from functions import getinfo
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances_argmin_min
from sklearn.metrics import pairwise_distances
from sklearn.preprocessing import StandardScaler

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns



In [ ]:
#Scrapeamos los supermercados.
supermercados_scrap = ['mercadona','dia','lidl','carrefour']
for super in supermercados_scrap:
    getinfo(super,'key')


In [74]:
'''#Añadimos los productos que dieron error porque nos quedamos sin créditos
productos = ['Bebida +Proteínas Sabor Coco', 'Atún en Aceite de Oliva', 'Patatas Chips',
    'Costillas de Cerdo Asadas', 'Pan de Molde sin Gluten', 'Hélices Vegetales',
    'Café Soluble Descafeinado', 'Chocolate Extrafino Negro 85% Cacao',
    'Barquillos Artesanos', 'Ternera al Ajillo', 'Yogur para Beber 0 Fresa']
porcion = ['100g', '60g', '100g', '100g', '100g', '100g', '2g', '20g', '100g', '100g', '100g']
kcal = ['48', '147', '506', '257', '282', '358', '5', '115', '401', '98', '31']
grasa = ['0.4', '10.8', '27.2', '15', '11', '1.5', '0', '9.2', '6.3', '2', '0.1']
grasa_saturada = ['0.2', '1.740', '3.9', '5.8', '1.3', '0.3', '0', '5.6', '1.9', '0.8', '0']
carbs = ['3.8', '0.54', '58', '16', '40', '72', '0.76', '4.3', '79', '1.5', '4.2']
azucar = ['3.5', '0.30', '1.6', '12', '3.8', '3.1', '0', '3.0', '39', '0.5', '4.1']
prote = ['7.2', '12.6', '7.3', '14', '1', '12', '0.34', '2.4', '7', '19', '3.2']
sal = ['0.12', '0', '0.01', '1.2', '1.3', '0.03', '0', '0', '0', '1.3', '0']
productos_fallo = pd.DataFrame({
    'Producto': productos,
    'Porcion': porcion,
    'Kcal': kcal,
    'Grasa': grasa,
    'Grasas saturadas': grasa_saturada,
    'Carbohidratos': carbs,
    'Azucares': azucar,
    'Proteina': prote,
    'Sal': sal
})
productos = ['Jamon serrano reserva','Choco Galleta-Biscoito','Frito Mallorquin']
porcion = ['100g','100g','100g']
kcal = ['248','501','122']
grasa = ['12.2','25','3.9']
grasa_saturada = ['4.6','14','1.9']
carbs = ['1','64','2.7']
azucar = ['0.5','45','0.9']
prote = ['33.5','5.1','18.9']
sal = ['3.6','0.44','0.37']

productos_fallodos = pd.DataFrame({
    'Producto': productos,
    'Porcion': porcion,
    'Kcal': kcal,
    'Grasa': grasa,
    'Grasas saturadas': grasa_saturada,
    'Carbohidratos': carbs,
    'Azucares': azucar,
    'Proteina': prote,
    'Sal': sal
})
'''

#Cargamos todos los dataframe:
Datamerca = pd.read_csv('Dataset/Dismissed Data/Scrap_Mercadona.csv')
DataCrf = pd.read_csv('Dataset/Dismissed Data/Scrap_CRF.csv')
DataDia = pd.read_csv('Dataset/Dismissed Data/Scrap_DIA.csv')
DataLidl = pd.read_csv('Dataset/Dismissed Data/Scrap_LILD.csv')



#Añadimos columna en cada Dataframe con su supermercado.
Datamerca['Supermercado'] = 'Mercadona'
DataLidl['Supermercado'] = 'Lidl'
DataDia['Supermercado'] = 'Dia'
DataCrf['Supermercado'] = 'Carrefour'

#Juntamos Dataframe, reseteamos index y dropeamos las columnas que generaron fallo.
data = pd.concat([Datamerca, DataCrf, DataDia, DataLidl])
data.reset_index(inplace = True, drop = True)
data.dropna(inplace = True)

#Quitamos todas las 'g'
#Subst los - por 0.
#pasamos a número
cols = list(data.columns)
cols
for i in cols[1:-1]:
    print(i)
    data[i] = data[i].str.replace('g', '')
    data[i] = data[i].str.replace('kcal','')
    data[i] = data[i].str.replace(',','.')
    data[i] = data[i].str.replace('-','0')
    data[i] = data[i].str.replace(' ','')
    data[i] = data[i].str.replace('ml','')

#Como hay porciones que son 1Galleta(25g) - nos quedamos con lo que hay entre parentesis i como hemos quitado los g pues ya lo tenemos.
porcion_num = []
for i in data['Porcion']:
    
    parentesis = re.search('\((.*?)\)',i)
    if parentesis:
        porcion_num.append(parentesis.group(1))
    else:
        porcion_num.append(i)

data['Porcion'] = porcion_num


#Sigue habiendo productos de los que no tenemos la info correcta: chekiamos cuales son
ture = []
for i in data['Porcion']:
    
    letras = re.search('\d\D\D|\D\D',i)
    if letras:
        ture.append(True)
    else:
        ture.append(False)

data['Porcion_check'] = ture

#Como son pocos, los cambiamos manualmente:
cols
for i in cols[1:-1]:
    data[i] = data[i].str.replace('escurrido', '52')
    data[i] = data[i].str.replace('escurrida', '62')
    data[i] = data[i].str.replace('1helado', '70')
    data[i] = data[i].str.replace('1porción','100')
    data[i] = data[i].str.replace('4unidades','100')
    data[i] = data[i].str.replace('4alletas','44')
    data[i] = data[i].str.replace('1unidad','100')
    data[i] = data[i].str.replace('2alletas','35')
    data[i] = data[i].str.replace('5alletas','55')
    data[i] = data[i].str.replace('1barrita','25')
    data[i] = data[i].str.replace('100oz','100')
    data[i] = data[i].str.replace('2tortitas','25')
    data[i] = data[i].str.replace('2unidades','32')
    data[i] = data[i].str.replace('3barritas','50')


#Ponemos en float los datos que nos interesan:
data.drop(columns='Porcion_check', inplace = True)
for i in cols[1:-1]:
    data[i] = data[i].astype(float)

#Convertimos todo a porcion 100g:
cols = list(data.columns)[2:-1]
for i in cols:
    data[i] = (data[i]/data['Porcion'])*100
data.drop(columns='Porcion', inplace = True) 
#Y nos cargamos la columna porción porque no nos hace falta más.

#Hay que quitar productos "Duplicados" hay varios productos con el mismo nombre en el mismo supermercado. Los hay que elminiar
dupli = data.duplicated(subset=['Producto','Supermercado'], keep = 'first')
dupli

data = data[~dupli]

#Guardamos el DataSet limpio por si las moscas:
data.to_excel(f'Dataset/Data.xlsx', index=False)


Porcion
Kcal
Grasa
Grasas saturadas
Carbohidratos
Azucares
Proteina
Sal


In [29]:
#data = data.drop(data[data['Porcion_check'] == True].index)

In [73]:
data['Kcal'].max()

1500.0

In [59]:
#data.drop(columns='Porcion_check', inplace = True)
cols = list(data.columns)[2:-1]
for i in cols:
    data[i] = (data[i]/data['Porcion'])*100

In [58]:
data.to_excel('Dataset/Data.xlsx', index=False)

In [ ]:
#Con el dataset total hacemos un KNN:
x=data[list(data.columns)[1:-1]]
scaler=StandardScaler()
x_prep=scaler.fit_transform(x)

K=range(2, 20)
inertia=[]
for k in K:
    kmeans=KMeans(n_clusters=k, random_state=42)
    kmeans.fit(x_prep)
    inertia.append(kmeans.inertia_)

plt.figure(figsize=(16,8))
plt.plot(K, inertia, "bx-")
plt.xlabel("k")
plt.ylabel("inertia")
plt.xticks(np.arange(min(K), max(K)+1, 1.0))
plt.show()

Diferenciamos dos codos, uno en el 6 y el otro en el 11

![](Images\Codo_dataset_con_sal_y_totales.png)

In [ ]:
#El 6 nos crea clústers demasiado generales. Así como tenemos los datos planteados nos interesa dividir bastante la muestra
#Con 11 clúster, los clúster están mejor pero hay un clúster que son productos con mucha sal que solo tiene 5 productos
kmeans = KMeans(n_clusters=11, random_state=123)
kmeans.fit(x_prep)
cluster = kmeans.predict(x_prep)
data['Cluster_Kmeans'] = cluster

data.Cluster_Kmeans.value_counts()



![](Images\Clsuter_con_sal_y_totales_11_cluster.png)

In [ ]:
#Probando con otros números de cluster la sal siempre tiene mucha importancia dentro de algún cluster y no me interesa que sea tan decisivio. 

#Quitamos la sal y nos salen mejor los cluster:
data.drop(columns='Sal', inplace = True)
x=data[list(data.columns)[1:-1]]
scaler=StandardScaler()
x_prep=scaler.fit_transform(x)

K=range(2, 20)
inertia=[]
for k in K:
    kmeans=KMeans(n_clusters=k, random_state=42)
    kmeans.fit(x_prep)
    inertia.append(kmeans.inertia_)

plt.figure(figsize=(16,8))
plt.plot(K, inertia, "bx-")
plt.xlabel("k")
plt.ylabel("inertia")
plt.xticks(np.arange(min(K), max(K)+1, 1.0))
plt.show()

![](Images\Codo_dataset_sin_sal_y_totales.png)

In [1]:
#Probamos con 7 codos y nos quedan unos clúster bastante acertados. 

In [2]:
#Definimos una funcion para poder comprobar cuáles son las predicciones para algunos productos y hacer pruebas

In [ ]:
#Hacer un VIF para la correlacion entre variables (Quitamos Kcal?)
corraus = data.drop(columns = ['Producto', 'Supermercado']).corr()
mask = np.triu(np.ones_like(corraus, dtype=bool))
corr_var = sns.heatmap(corraus,annot=True,mask=mask)
plt.title("Correlación")
plt.show()
#Kcal esta bastante correlacionado con la variable grasas y Carbohidratos. Probaremos hacer el KNN sin ella.


#Probar otros modelos

#Probar como predice con los datos porcentuales. 
data['Saturada%'] = data['Grasas saturadas']/data['Grasa']*100
data['Azucares%'] = data['Azucares']/data['Carbohidratos']*100

data.drop(columns=['Azucares', 'Grasas saturadas'], inplace = True)
data_numeric = data.apply(pd.to_numeric, errors = 'coerce')
masca = np.isinf(data_numeric).any(axis=1)

infinitos = data_numeric[masca]
infinitos

data.replace([np.inf, -np.inf], 0, inplace=True)
data = data.fillna(0)

#Hacer clúster de clústers? -->Así organizamos según macronutrientes y dentro de estos distinguimos tendencias. 


